# Lectura inicial de un archivo a ver qué tiene

In [33]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

conf = (SparkConf().set("spark.executor.memory", "6g")   #<--- Por executor. Como hay dos instancias, la mitad por executor.
                    .set("spark.executor.cores", "3")    #<--- Por cada executor. Como hay 4, 4*3=12 cores se usan.
                    .set("spark.executor.instances", "2")
                    .set("spark.eventLog.enabled", "true"))
spark = SparkSession.builder.master("spark://spark-master:7077").appName("Ecommerce").config(conf=conf).getOrCreate()
spark

In [34]:
%%time
from pyspark.sql.types import StructType
from pyspark.sql.types import StringType,BooleanType,DateType,IntegerType
import pyspark.sql.functions as F
from pyspark.sql.functions import *


proc='*'
""" PARQUET
df = spark.read.option("mergeSchema", True).option("basePath", "file:///opt/workspace/datos/ecommerce/ecommerce.parquet/")\
               .format("parquet").load(['./datos/ecommerce/ecommerce.parquet/proc='+proc])
"""
#CSV

df = spark.read.csv("./Fotolescu/useful-master/kaggle/coronavirus-forecasting/data/train.csv", sep=",", header=True, inferSchema=True)



# shape:
print(80*'=','\nfilas --> ', df.count(), 'columnas --> ', len(df.columns), '\n', 80*'=')
df.show(truncate=False)

#df = df.withColumn("proc", substring("ctId",1,6))
#df.write.mode("append").partitionBy("proc").parquet("./datos/fakeContracts01.parquet")

filas -->  16756 columnas -->  8 
+---+--------------+--------------+----+----+----------+--------------+----------+
|Id |Province/State|Country/Region|Lat |Long|Date      |ConfirmedCases|Fatalities|
+---+--------------+--------------+----+----+----------+--------------+----------+
|1  |null          |Afghanistan   |33.0|65.0|2020-01-22|0.0           |0.0       |
|2  |null          |Afghanistan   |33.0|65.0|2020-01-23|0.0           |0.0       |
|3  |null          |Afghanistan   |33.0|65.0|2020-01-24|0.0           |0.0       |
|4  |null          |Afghanistan   |33.0|65.0|2020-01-25|0.0           |0.0       |
|5  |null          |Afghanistan   |33.0|65.0|2020-01-26|0.0           |0.0       |
|6  |null          |Afghanistan   |33.0|65.0|2020-01-27|0.0           |0.0       |
|7  |null          |Afghanistan   |33.0|65.0|2020-01-28|0.0           |0.0       |
|8  |null          |Afghanistan   |33.0|65.0|2020-01-29|0.0           |0.0       |
|9  |null          |Afghanistan   |33.0|65.0|2020-01-

In [3]:
df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- Province/State: string (nullable = true)
 |-- Country/Region: string (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Long: double (nullable = true)
 |-- Date: string (nullable = true)
 |-- ConfirmedCases: double (nullable = true)
 |-- Fatalities: double (nullable = true)



### Damos un vistazo rápido al contenido del archivo

In [4]:
df.describe().show()

+-------+-----------------+--------------+--------------+------------------+-----------------+----------+------------------+------------------+
|summary|               Id|Province/State|Country/Region|               Lat|             Long|      Date|    ConfirmedCases|        Fatalities|
+-------+-----------------+--------------+--------------+------------------+-----------------+----------+------------------+------------------+
|  count|            16756|          7670|         16756|             16697|            16697|     16756|             16756|             16756|
|   mean|          13189.5|          null|          null| 26.33732402826862|5.039602826854996|      null| 271.7501193602292| 9.221532585342564|
| stddev|7624.686931555773|          null|          null|22.963062991020543|79.93526713077237|      null|3261.1523047726537|141.52458182865337|
|    min|                1|       Alabama|   Afghanistan|          -41.4545|        -157.4983|2020-01-22|               0.0|            

In [5]:
%%time
for c in df.columns:df.groupBy(c).count().show(truncate=False)

+----+-----+
|Id  |count|
+----+-----+
|148 |1    |
|471 |1    |
|496 |1    |
|1238|1    |
|1342|1    |
|1591|1    |
|1959|1    |
|2142|1    |
|2366|1    |
|2659|1    |
|3175|1    |
|3749|1    |
|3918|1    |
|4101|1    |
|4519|1    |
|4935|1    |
|5156|1    |
|5518|1    |
|5803|1    |
|6336|1    |
+----+-----+
only showing top 20 rows

+----------------------------+-----+
|Province/State              |count|
+----------------------------+-----+
|Utah                        |59   |
|Hawaii                      |59   |
|Manitoba                    |59   |
|Guangdong                   |59   |
|Minnesota                   |59   |
|Ohio                        |59   |
|Hunan                       |59   |
|Shanxi                      |59   |
|Tibet                       |59   |
|Hubei                       |59   |
|United States Virgin Islands|59   |
|Tianjin                     |59   |
|Northern Territory          |59   |
|Beijing                     |59   |
|Arkansas                    |59 

### Cuenta cuántos nan or null hay en cada columna:

In [6]:
print(df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show())

+---+--------------+--------------+---+----+----+--------------+----------+
| Id|Province/State|Country/Region|Lat|Long|Date|ConfirmedCases|Fatalities|
+---+--------------+--------------+---+----+----+--------------+----------+
|  0|          9086|             0| 59|  59|   0|             0|         0|
+---+--------------+--------------+---+----+----+--------------+----------+

None


In [3]:
df.filter(col('Province/State').isNull()).groupBy('Country/Region').count().show()

+--------------+-----+
|Country/Region|count|
+--------------+-----+
|      Paraguay|   59|
|        Russia|   59|
|       Senegal|   59|
|        Sweden|   59|
|        Guyana|   59|
|        Jersey|   59|
|   Philippines|   59|
|      Djibouti|   59|
|      Malaysia|   59|
|     Singapore|   59|
|        Turkey|   59|
|          Iraq|   59|
|       Germany|   59|
|   Afghanistan|   59|
|      Cambodia|   59|
|        Jordan|   59|
|      Maldives|   59|
|        Rwanda|   59|
|         Sudan|   59|
|        Greece|   59|
+--------------+-----+
only showing top 20 rows



In [44]:
df.filter(col('Country/Region') == 'Paraguay').filter(col('ConfirmedCases') == 11.).show(59)

+-----+--------------+--------------+--------+--------+----------+--------------+----------+
|   Id|Province/State|Country/Region|     Lat|    Long|      Date|ConfirmedCases|Fatalities|
+-----+--------------+--------------+--------+--------+----------+--------------+----------+
|16332|          null|      Paraguay|-23.4425|-58.4438|2020-03-18|          11.0|       0.0|
|16333|          null|      Paraguay|-23.4425|-58.4438|2020-03-19|          11.0|       0.0|
+-----+--------------+--------------+--------+--------+----------+--------------+----------+



In [13]:
%%time
spark.sparkContext.stop()


23/02/23 18:19:30 WARN NioEventLoop: Selector.select() returned prematurely 512 times in a row; rebuilding Selector io.netty.channel.nio.SelectedSelectionKeySetSelector@4ea20da7.


CPU times: user 3.81 ms, sys: 0 ns, total: 3.81 ms
Wall time: 278 ms
